In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip3 install -q ktrain

     |████████████████████████████████| 421.8MB 42kB/s 
     |████████████████████████████████| 6.7MB 20.8MB/s 
     |████████████████████████████████| 983kB 18.2MB/s 
     |████████████████████████████████| 245kB 42.5MB/s 
     |████████████████████████████████| 675kB 34.0MB/s 
     |████████████████████████████████| 471kB 38.9MB/s 
     |████████████████████████████████| 450kB 38.3MB/s 
     |████████████████████████████████| 3.9MB 39.3MB/s 
     |████████████████████████████████| 890kB 37.7MB/s 
     |████████████████████████████████| 3.8MB 38.5MB/s 
     |████████████████████████████████| 1.1MB 40.6MB/s 
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
train_b=pd.read_csv("/content/drive/My Drive/minor/english_dataset/eng23train.csv") #training data
test_b=pd.read_csv("/content/drive/My Drive/minor/english_dataset/eng23test.csv") #testing data
x_train,x_test,y_train,y_test=train_test_split(train_b['text'],train_b['task_3'],test_size=0.15,random_state=42) #training and validation split
target_names=[0,1] #names of classes

In [ ]:
#convertig the tweets to list of strings and labels to list data type
x_train=list(x_train)
x_test=list(x_test)
y_train=list(y_train)
y_test=list(y_test)

In [ ]:
import ktrain
from ktrain import text
MODEL_NAME = 'xlnet-base-cased' #xlnet predefined model
t = text.Transformer(MODEL_NAME, maxlen=500, classes=target_names) #load model, text and label preprocessor for xlnet
trn = t.preprocess_train(x_train, y_train) #preprocess text and labels for training data
val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data
model = t.get_classifier() #get xlnet classifier
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6) #get trainable learner for xlnet

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:398: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


preprocessing train...
language: en
train sequence lengths:
	mean : 25
	95percentile : 47
	99percentile : 53


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 23
	95percentile : 45
	99percentile : 52


In [ ]:
learner.fit_onecycle(5e-5, 4) #training for epochs=4 ,learning rate=5e-5



begin training using onecycle policy with max lr of 5e-05...
Train for 321 steps, validate for 11 steps
Epoch 1/4
321/321 [==============================] - 983s 3s/step - loss: 0.3231 - accuracy: 0.9006 - val_loss: 0.2613 - val_accuracy: 0.9118
Epoch 2/4
321/321 [==============================] - 963s 3s/step - loss: 0.3115 - accuracy: 0.9011 - val_loss: 0.2452 - val_accuracy: 0.9118
Epoch 3/4
321/321 [==============================] - 965s 3s/step - loss: 0.2686 - accuracy: 0.8980 - val_loss: 0.2397 - val_accuracy: 0.9118
Epoch 4/4
321/321 [==============================] - 965s 3s/step - loss: 0.1767 - accuracy: 0.9438 - val_loss: 0.2629 - val_accuracy: 0.9029


In [ ]:
learner.validate() #predictions metrics for validation data

              precision    recall  f1-score   support

           0       0.45      0.43      0.44        30
           1       0.95      0.95      0.95       310

    accuracy                           0.90       340
   macro avg       0.70      0.69      0.69       340
weighted avg       0.90      0.90      0.90       340



array([[ 13,  17],
       [ 16, 294]])

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t) #get predictor for the trained model of  xlnet

In [ ]:
test_b_text=list(test_b['text']) #get text for testing
test_b_label=list(test_b['task_3']) #get label of testing

In [ ]:
# predicted probability scores for each category
pred_y=predictor.predict_proba(test_b_text)

In [ ]:
pred_y=np.argmax(pred_y,axis=1) #get class with maximum probability

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(test_b_label,pred_y)) # classification report

              precision    recall  f1-score   support

           0       0.21      0.30      0.25        43
           1       0.87      0.80      0.83       245

    accuracy                           0.73       288
   macro avg       0.54      0.55      0.54       288
weighted avg       0.77      0.73      0.74       288

